In [1]:
import numpy as np
import pandas as pd
import pyodbc
import seaborn as sns
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import math
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from bokeh.io import output_notebook, show, curdoc, output_file,push_notebook
from bokeh.layouts import widgetbox, row, column
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, Slider, CDSView, GroupFilter,BooleanFilter
from bokeh.models.widgets import CheckboxGroup, TextInput, Button
from bokeh.models.widgets.markups import Paragraph 
from bokeh.core.properties import value
#from bokeh.transform import factor_cmap
#from bokeh.palettes import all_palettes

online = 0
if online == 1:
    conn=pyodbc.connect('Driver={SQL Server};'
                          'Server=balt-sql-fc;'
                          'Database=CitiStat;'
                          'Trusted_Connection=yes;')

In [2]:
def due_date(create_date,sla):
    due = dt.datetime.strptime(create_date, '%Y-%m-%d').date() + dt.timedelta(days=sla)
    return str(due)
def lower_str(text):
    return text.lower()
def textstr2date(text):
    if pd.notnull(text):
        new_str=str(text)[0:10]
    else:
        new_str = text
    return new_str
def oopen(data_inp,date_comp):
    #COUNTS NUMBER OPEN
    count_close=len(data_inp[(data_inp['Date_Create']<date_comp)&((data_inp['Status'].apply(lower_str).str.contains('close'))&(data_inp['Date_Finish']>date_comp))])
    count_cancel=len(data_inp[(data_inp['Date_Create']<date_comp)&((data_inp['Status'].apply(lower_str).str.contains('cancel'))&(data_inp['Date_Finish']>date_comp))])
    count_new =len(data_inp[(data_inp['Date_Create']<date_comp)&(data_inp['Status'].apply(lower_str).str.contains('new'))])
    counter=count_close+count_new#+count_cancel
    #Doesn't count PENDING
    return counter

def oover(data_inp,date_comp):
    #COUNTS NUMBER OPEN
    count_close=len(data_inp[(data_inp['Due_date']<date_comp)&((data_inp['Status'].apply(lower_str).str.contains('close'))&(data_inp['Date_Finish']>date_comp))])
    count_cancel=len(data_inp[(data_inp['Due_date']<date_comp)&((data_inp['Status'].apply(lower_str).str.contains('cancel'))&(data_inp['Date_Finish']>date_comp))])
    count_new =len(data_inp[(data_inp['Due_date']<date_comp)&(data_inp['Status'].apply(lower_str).str.contains('new'))])
    counter=count_close+count_new#+count_cancel
    #Doesn't count PENDING
    return counter

def days_between(d1, d2):
    d1 = dt.datetime.strptime(d1, "%Y-%m-%d")
    d2 = dt.datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)
#should be attendance rate, need to change every instance in code
def absence(absence_rate,no_crews):
    a = np.random.binomial(1, absence_rate, no_crews)
    return a

def wo_creation(date,quad):
    month = date.month
    avg_work = wo_creation_hist.loc[(wo_creation_hist['quad']==quad)&(wo_creation_hist['month']==month),'avg_wo'].values
    sd_work = wo_creation_hist.loc[(wo_creation_hist['quad']==quad)&(wo_creation_hist['month']==month),'sd_wo'].values
    wo = math.ceil(np.random.normal(avg_work,sd_work))
    return wo
def add_years(d, years):
    """Return a date that's `years` years after the date (or datetime)
    object `d`. Return the same calendar date (month and day) in the
    destination year, if it exists, otherwise use the following day
    (thus changing February 29 to March 1).

    """
    try:
        return d.replace(year = d.year + years)
    except ValueError:
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))

In [3]:
def simulation():
    t_dquad =data_quad.copy()
    data_quad['vendor_budget'] = 0
    for j in quads:
        temp = t_dquad[t_dquad['quad']==j].reset_index()
        backlog = temp['backlog'].values[0]*(1-temp['cancel_rate'].values)
        quad_sys_backlog = [backlog]
        for i in range(1,len(dates)):
            if add_wo == 1:
                work_created = wo_creation(quad_backlog.loc[i,'Date'],j)
                work_created = work_created#*(1-temp['cancel_rate'].values)
            elif add_wo == 0:
                work_created = 0
            #0 indexed, so Monday=0,and Sunday =6. If saturday crews then working_days=6
            if quad_backlog.loc[i,'Date'].weekday() < working_days:
                #crews4day= absence(temp['absence_rate'],temp['crew_size'])
                crews4day = temp['crew_size'].values
                month = quad_backlog.loc[i,'Date'].month
                #crew_rate = wo_crew_rate.loc[wo_crew_rate['month']==month,'avg_rate'].values
                crew_rate = temp['rate'].values
                if j == 'Clean Size 1':
                    if sum(data_quad['vendor_budget']) < 300000:
                        #vendor = 21
                        vendor = math.ceil(temp['vendor'].values/len(dates))
                        data_quad.loc[data_quad['quad']==j,'vendor_budget'] = data_quad.loc[data_quad['quad']==j,'vendor_budget']  + vendor*160
                elif j == 'Clean Size 2':
                    if sum(data_quad['vendor_budget']) < 300000:
                        #vendor = 11
                        vendor = math.ceil(temp['vendor'].values/len(dates))
                        data_quad.loc[data_quad['quad']==j,'vendor_budget']  = data_quad.loc[data_quad['quad']==j,'vendor_budget']  + vendor*180
                else: 
                    vendor = 0
                work_completed = sum(crews4day*crew_rate)+vendor
                backlog = backlog-work_completed+work_created
                if backlog < 0:
                    backlog=0
                quad_sys_backlog.append(backlog)
            else:
                backlog = backlog+work_created
                quad_sys_backlog.append(backlog)
        quad_backlog[j] = quad_sys_backlog
    quad_backlog['Total'] = quad_backlog['Clean Size 1']+quad_backlog['Clean Size 2']+quad_backlog['Clean Size 3']
    return quad_backlog

In [4]:
if online == 1:
    init_back =  pd.read_sql('''select quad, 
    count([Record ID]) as backlog
    from (select [Record ID], 
        case when [Status] like 'CLOSE%' then 'Closed'
                when [Status] like 'CANCEL%' then 'Cancelled'
                WHEN [Date Finish] is null THEN 'Open'
                end as complete_status,
                [Clean Size],
        case when [Solid Waste Quadrant] = 'NORTHEAST' THEN 'NE'
            WHEN [Solid Waste Quadrant] = 'NORTHWEST' THEN 'NW'
            WHEN [Solid Waste Quadrant] = 'SOUTHEAST' THEN 'SE'
            WHEN [Solid Waste Quadrant] = 'SOUTHWEST' THEN 'SW'
            END AS	quad
    from CitiStat.dbo.CHIP_WorkOrders
    where 1=1
    and [Work Order Type]='Cleaning'
    and [Date Finish] is null
    and [Solid Waste Quadrant] in ('NORTHEAST','SOUTHEAST','SOUTHWEST','NORTHWEST')
    ) foo
    where 1=1
    group by quad
    ''',conn)
    init_back.to_csv('initial_backlog.csv')
    
    wo_creation_hist = pd.read_sql('''
    select month(date_create) month, quad, avg(wo_count) avg_wo, stdev(wo_count) sd_wo
    from
    (select [date_create], quad, count([Record ID]) wo_count
    from (
    select  [Record ID], 
            cast([Date Create] as DATE) as date_create,
            [Clean Size],
        case when [Solid Waste Quadrant] = 'NORTHEAST' THEN 'NE'
            WHEN [Solid Waste Quadrant] = 'NORTHWEST' THEN 'NW'
            WHEN [Solid Waste Quadrant] = 'SOUTHEAST' THEN 'SE'
            WHEN [Solid Waste Quadrant] = 'SOUTHWEST' THEN 'SW'
            END AS	quad
    from CitiStat.dbo.CHIP_WorkOrders
    where 1=1
    and [Work Order Type]='Cleaning'
    and [Date Create] between '2019-01-01' and '2020-01-01'
    and [Solid Waste Quadrant] in ('NORTHEAST','SOUTHEAST','SOUTHWEST','NORTHWEST')
    ) foo
    where 1=1
    group by date_create, quad
    ) foo
    where 1=1
    --and wo_count > 10
    group by month(date_create),quad
    order by month, quad''',conn)
    wo_creation_hist.to_csv('historical_creation.csv')
    
    wo_crew_rate = pd.read_sql('''
    select month(date_finish) month, 
            --quad,
            --clean_size,
            avg(crew_count) avg_crew,
            avg(wo_count) avg_wo, 
            stdev(wo_count) sd_wo,
            avg(wo_count)/avg(crew_count) avg_rate
    from
    (
    select [date_finish], 
            --quad,
            --clean_size,
            count(distinct crew_name) as crew_count,
            count([Record ID]) wo_count,
            count([Record ID])/count(distinct crew_name) as rate
    from (
    select  [Record ID], 
            cast([Date Finish] as DATE) as date_finish,
            [Clean Size] as clean_size,
            [Crew Name] as crew_name,
        case when [Solid Waste Quadrant] = 'NORTHEAST' THEN 'NE'
            WHEN [Solid Waste Quadrant] = 'NORTHWEST' THEN 'NW'
            WHEN [Solid Waste Quadrant] = 'SOUTHEAST' THEN 'SE'
            WHEN [Solid Waste Quadrant] = 'SOUTHWEST' THEN 'SW'
            END AS	quad
    from CitiStat.dbo.CHIP_WorkOrders
    where 1=1
    and [Work Order Type]='Cleaning'
    and [Date Finish] between '2019-01-01' and '2020-01-01'
    and [Solid Waste Quadrant] in ('NORTHEAST','SOUTHEAST','SOUTHWEST','NORTHWEST')
    and lower([Status]) like 'close%'
    ) foo
    where 1=1
    group by date_finish--, clean_size--, quad
    ) foo
    where 1=1
    and wo_count > 30
    group by month(date_finish)--,clean_size--,quad
    order by month--,clean_size--, quad
    ''', conn)
    wo_crew_rate.to_csv('wo_crew_rate.csv')
    data_quad = data_quad.merge(init_back,how='left', on ='quad' )
if online == 0:
    #init_back = pd.read_csv('initial_backlog.csv')
    wo_creation_hist = pd.read_csv('historical_cs_creation.csv')
    #wo_crew_rate = pd.read_csv('wo_cs_crew_rate.csv')

In [5]:
quads = ['Clean Size 1','Clean Size 2','Clean Size 3']
data_quad=pd.DataFrame({
    'quad'       : quads,
    'crew_size'  : [4,6,6],
    'rate' : [10,8,3],
    #'worksite_diff' : [1,0.8,1.2,0.75],
    'worksite_diff' : [1,1,1],
    #'absence_rate': [0.9,0.83,0.9,0.78]
    'absence_rate': [1,1,1],
    'backlog':[1162,2331,810],
    'vendor':[1250,555,0],
    'vendor_budget':[0,0,0],
    'cancel_rate': [.1,.1,.1]
})


saturdays = 1
working_days = 5 + saturdays

In [6]:
a = datetime.today()
b = pd.to_datetime(dt.date(2020,3,31))
dates =  pd.to_datetime(a.strftime('%Y-%m-%d')) + pd.to_timedelta(np.arange((b-a).days), 'D')
add_wo = 1
quad_backlog = pd.DataFrame(dates)
quad_backlog.columns = ['Date']
quad_backlog = simulation()

In [7]:
#fig = plt.figure()
#ax = plt.subplot(111)

#for i in ['Clean Size 1','Clean Size 2','Clean Size 3','Total']:
#    if i == 'Total':
#        ax.plot(quad_backlog['Date'],quad_backlog[i])
#    else:
#        ax.plot(quad_backlog['Date'],quad_backlog[i],alpha = .5)
####plt.plot(df_clean.loc[df_clean['Date']>'2019-01-01','Date'],df_clean.loc[df_clean['Date']>'2019-01-01','Open_WO'])
#### Shrink current axis by 20%
#box = ax.get_position()
#ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
#plt.title("Backlog Size")
#plt.ylabel("Backlog Size")
#plt.xlabel("Date")
#plt.xticks(rotation=45) 

# Put a legend to the right of the current axis
#ax.legend(['Clean Size 1','Clean Size 2','Clean Size 3','Total'],title= 'Clean Size',loc='center left', bbox_to_anchor=(1, 0.5))

#plt.show()

In [8]:
##############Working Widgets
#Sliders
def callback_slide1(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 1','crew_size'] = slider_size1.value   
def callback_slide2(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 2','crew_size'] = slider_size2.value
def callback_slide3(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 3','crew_size'] = slider_size3.value
def callback_rate1(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 1','rate'] = slider_rate1.value   
def callback_rate2(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 2','rate'] = slider_rate2.value
def callback_rate3(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 3','rate'] = slider_rate3.value
#Buttons
def submit():
    source.data = simulation()  
#Text Input
def callback_vendor1(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 1','vendor'] = int(vendor_s1_input.value_input)
def callback_vendor2(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 2','vendor'] = int(vendor_s2_input.value_input)

##############In-Progress Widgets
def callback_cancel(attr, old, new):
    data_quad.loc[data_quad['quad']=='Clean Size 1','cancel_rate'] = slider_cancel.value
    data_quad.loc[data_quad['quad']=='Clean Size 2','cancel_rate'] = slider_cancel.value
    data_quad.loc[data_quad['quad']=='Clean Size 3','cancel_rate'] = slider_cancel.value
#############Not Working Widgets


def callback_sat():
    if saturday_button.active[0] == True:
        saturdays = 1
    else:
        saturday = 0
    working_days = 5 + saturdays

In [9]:
vendor_budget = pd.DataFrame({
    'vendor': ['Evergreen'],
    'Clean Size 1': [data_quad.loc[data_quad['quad']=='Clean Size 1','vendor_budget'].values[0]],
    'Clean Size 2': [data_quad.loc[data_quad['quad']=='Clean Size 2','vendor_budget'].values[0]]
})
grouped = vendor_budget.groupby('vendor')['Clean Size 1', 'Clean Size 2'].sum()
grouped = grouped / 1000

In [10]:
source = ColumnDataSource(quad_backlog)
source_vendor = ColumnDataSource(grouped)
vendor_list = source_vendor.data['vendor'].tolist()
# Create a figure with x_axis_type="datetime": p
p = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label='Backlog',y_range = [0,5500])
           #,x_range=[a,(pd.to_datetime(a.strftime('%Y-%m-%d')) + pd.to_timedelta((b-a).days, 'D'))])

p.line('Date','Total',source=source, color='red',legend=value('Total'))
p.line('Date','Clean Size 1',alpha = 1,source=source, color='orange',legend=value("Clean 'Size 1'"))
p.line('Date','Clean Size 2',alpha = 1,source=source, color='blue',legend=value("Clean 'Size 2'"))
p.line('Date','Clean Size 3',alpha = 1,source=source, color='green',legend=value("Clean 'Size 3'"))
p.legend.background_fill_color='ghostwhite'


b = figure(x_axis_label='Vendor', y_axis_label='Vendor Budget Used (000)',x_range = vendor_list,y_range = [0,300],
           plot_width=300, plot_height=300)
b.vbar_stack(stackers=['Clean Size 1', 'Clean Size 2'],
             x='vendor', source=source_vendor,
             legend = [value('Clean Size 1'), value('Clean Size 2')],
             width=0.5,color=['orange','blue'])
b.legend.orientation = "horizontal"
b.legend.location = "bottom_center"

slider_width = 160
# Create a slider: slider
slider_size1 = Slider(title="Clean 'Size 1' Crews", start=0, end=10, step=1, value=4,width = slider_width)
slider_size1.on_change('value',callback_slide1)
slider_size2 = Slider(title="Clean 'Size 2' Crews", start=0, end=10, step=1, value=6,width = slider_width)
slider_size2.on_change('value',callback_slide2)
slider_size3 = Slider(title="Clean 'Size 3' Crews", start=0, end=10, step=1, value=6,width = slider_width)
slider_size3.on_change('value',callback_slide3)
slider_cancel = Slider(title='% of Work Orders Cancelled', start=0, end=.35, step=.01, value=.1,format='0%')
slider_cancel.on_change('value',callback_cancel)


# Create a slider: slider
slider_rate1 = Slider(title="Clean 'Size 1' Rate/Day", start=0, end=20, step=1, value=10,width = slider_width)
slider_rate1.on_change('value',callback_rate1)
slider_rate2 = Slider(title="Clean 'Size 2' Rate/Day", start=0, end=20, step=1, value=8,width = slider_width)
slider_rate2.on_change('value',callback_rate2)
slider_rate3 = Slider(title="Clean 'Size 3' Rate/Day", start=0, end=10, step=1, value=3,width = slider_width)
slider_rate3.on_change('value',callback_rate3)

######Checkbox
#saturday_button = CheckboxGroup(labels=["Saturday Crews"], active=[0])
#saturday_button.on_click(callback_sat)
#incoming_cancel = CheckboxGroup(labels=['Backlog work canceled?',"Incoming work cancelled?"], active=[0,1])

#Text Inputs
vendor_s1_input = TextInput(value='1250', title="Evergreen Total 'Size 1':",width=slider_width)
vendor_s1_input.on_change('value',callback_vendor1)
vendor_s2_input = TextInput(value='555', title="Evergreen Total 'Size 2':",width=slider_width)
vendor_s2_input.on_change('value',callback_vendor2)

# Create a widgetbox layout: layout
slide1 = widgetbox(slider_size1)
slide2 = widgetbox(slider_size2)
slide3 = widgetbox(slider_size3)
rate1 = widgetbox(slider_rate1)
rate2 = widgetbox(slider_rate2)
rate3 = widgetbox(slider_rate3)

slidecancel=widgetbox(slider_cancel)

#Button
button = Button(label="Submit", button_type="success")
button.on_click(submit)
#Paragraph
#test = Paragraph(text= str(data_quad.loc[data_quad['quad']=='Clean Size 3','crew_size'].values))
# Add the layout to the current document
curdoc().add_root(row(column(row(slide1,rate1),row(slide2,rate2),row(slide3,rate3),
#                             saturday_button,
                             slidecancel,
                             #incoming_cancel,
                            row(vendor_s1_input, vendor_s2_input),
                          button,
#                             b
                            ), p))

#output_file('line.html')
#show(row(column(row(slide1,rate1),row(slide2,rate2),row(slide3,rate3),
 #                           saturday_button,
#                            slidecancel,
#                            incoming_cancel,
#                            row(vendor_s1_input, vendor_s2_input),
#                            button,
#                            b),
#                            p))